In [190]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM, Dense, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf

from tensorflow.keras.layers import *

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam


import keras
from keras import layers

%matplotlib inline

# Preparing Data

In [191]:
df = pd.read_pickle('Lines.pkl')
df

,temp_max,temp_min,humidity,precip,wind_speed,degree,id,best_neighborID,pairs,array
34,21.5,7.6,37.9,0.0,24.1,28.7,34,39,"(34, 39)","[43128, 43355, 43430, 43500, 43570, 43676, 437..."
37,21.7,8.4,41.2,0.0,7.9,9.4,37,38,"(37, 38)","[44354, 44530, 44678, 44774, 44809, 44764, 446..."
38,21.7,7.3,35.2,0.0,6.5,5.0,38,37,"(38, 37)","[43005, 43036, 43031, 42931, 42764, 42572, 424..."
39,21.5,7.6,37.9,0.0,24.1,28.7,39,34,"(39, 34)","[42023, 42148, 42356, 42565, 42759, 42938, 431..."
40,21.7,7.8,35.6,0.0,23.8,28.4,40,45,"(40, 45)","[43793, 43827, 43837, 43826, 43814, 43781, 437..."
...,...,...,...,...,...,...,...,...,...,...
641,19.9,4.5,26.6,0.0,23.0,31.2,641,640,"(641, 640)","[42585, 42332]"
642,22.0,7.5,23.1,0.0,25.2,25.1,642,645,"(642, 645)","[43060, 43095, 43139, 43199, 43290, 43324, 432..."
645,22.0,7.5,23.1,0.0,25.2,25.1,645,642,"(645, 642)","[42998, 42914, 42820, 42724, 42687, 42709, 427..."
668,22.8,3.4,24.5,0.0,11.2,27.5,668,669,"(668, 669)","[42829, 42919, 42964, 42972, 42993, 42977, 429..."


In [192]:
data = df["array"].tolist()

raw_inputs = []
for i in range(len(data)):
    raw_inputs.append(data[i].tolist())

In [193]:
# Prepare input and target datasets
x, y = [], []
for i in range(len(raw_inputs)):
    x.append(raw_inputs[i][:-1])
    y.append(raw_inputs[i][-1])

In [194]:
x_padded_inputs = tf.keras.utils.pad_sequences(x, padding="post")
x_padded_inputs

array([[43128, 43355, 43430, ...,     0,     0,     0],
       [44354, 44530, 44678, ...,     0,     0,     0],
       [43005, 43036, 43031, ...,     0,     0,     0],
       ...,
       [42998, 42914, 42820, ...,     0,     0,     0],
       [42829, 42919, 42964, ...,     0,     0,     0],
       [43155, 43148, 43148, ...,     0,     0,     0]], dtype=int32)

In [195]:
n_features = 1
X = x_padded_inputs.reshape((x_padded_inputs.shape[0], x_padded_inputs.shape[1], n_features))
y = np.array(y)

In [196]:
X.shape, y.shape

((74, 56, 1), (74,))

In [197]:
X_train, y_train = X[:55], y[:55]
#X_val, y_val = X[50:55], y[50:55]
X_test, y_test = X[55:], y[55:]
X_train.shape, y_train.shape, X_test.shape, y_test.shape #X_val.shape, y_val.shape,

((55, 56, 1), (55,), (19, 56, 1), (19,))

# Model1

In [260]:
#model1 = keras.Sequential(layers.Embedding(input_dim=46281, output_dim=16, mask_zero=True, input_length=56))

model1 = keras.Sequential()
model1.add(Masking(mask_value=0.0, input_shape = (56,1))) # masking to ignore padded 0
model1.add(layers.LSTM(50, # check info on units here https://tung2389.github.io/coding-note/unitslstm
                      activation='relu'))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

In [261]:
model1.layers

In [262]:
model1.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_20 (Masking)        (None, 56, 1)             0         
                                                                 
 lstm_22 (LSTM)              (None, 50)                10400     
                                                                 
 dense_30 (Dense)            (None, 8)                 408       
                                                                 
 dense_31 (Dense)            (None, 1)                 9         
                                                                 
Total params: 10817 (42.25 KB)
Trainable params: 10817 (42.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [263]:
model1.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [264]:
history = model1.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=300,
    verbose= 1)

Epoch 1/300
2/2 [==============================] - 1s 182ms/step - loss: 2229530112.0000 - accuracy: 0.0000e+00 - val_loss: 2126816000.0000 - val_accuracy: 0.0000e+00
Epoch 2/300
2/2 [==============================] - 0s 31ms/step - loss: 2293500928.0000 - accuracy: 0.0000e+00 - val_loss: 1907199488.0000 - val_accuracy: 0.0000e+00
Epoch 3/300
2/2 [==============================] - 0s 29ms/step - loss: 2011353216.0000 - accuracy: 0.0000e+00 - val_loss: 1896123264.0000 - val_accuracy: 0.0000e+00
Epoch 4/300
2/2 [==============================] - 0s 29ms/step - loss: 1875161472.0000 - accuracy: 0.0000e+00 - val_loss: 1783183488.0000 - val_accuracy: 0.0000e+00
Epoch 5/300
2/2 [==============================] - 0s 28ms/step - loss: 1854744192.0000 - accuracy: 0.0000e+00 - val_loss: 1747522432.0000 - val_accuracy: 0.0000e+00
Epoch 6/300
2/2 [==============================] - 0s 29ms/step - loss: 1755726208.0000 - accuracy: 0.0000e+00 - val_loss: 2188648192.0000 - val_accuracy: 0.0000e+00
Epo

In [265]:
input_sequence = X_test[0]
input_sequence = input_sequence.reshape((1, x_padded_inputs.shape[1], n_features))

In [266]:
#input_sequence = X_train[-10]
#predicted_value = model1.predict(input_sequence)[0][-1]
#print("Predicted Value:", round(predicted_value, ndigits=2))

In [267]:
model1.predict(input_sequence)

1/1 [==============================] - 0s 118ms/step


array([[37346.26]], dtype=float32)

In [268]:
train_predictions = model1.predict(X_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions':np.round(train_predictions), 'Actuals':y_train})
train_results

2/2 [==============================] - 0s 5ms/step


,Train Predictions,Actuals
0,17322.0,42023
1,63856.0,43005
2,20536.0,44354
3,19197.0,43128
4,43582.0,45136
5,39470.0,43793
6,47079.0,45029
7,45146.0,44357
8,32633.0,44155
9,39646.0,44404


**Next steps:**
1. More Data (!)
2. Normalization
